In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [2]:
#========
# Setup
#========
df = pd.DataFrame({
    "order_id": [104 , 101 , 106 , 102 , 105 , 103] ,
    "region": ["West" , "East" , "West" , "East" , "West" , "East"] ,
    "product": ["Cable" , "Wire" , "Wire" , "Cable" , "Cable" , "Wire"] ,
    "month": ["2025-12" , "2025-11" , "2025-12" , "2025-12" , "2025-11" , "2025-11"] ,
    "sales": [110 , 120 , 95 , 150 , 90 , 80] ,
    "returns": [6 , 5 , np.nan , 4 , 3 , 2] ,
})
df["month"] = pd.to_datetime(df["month"] + "-01")
df

,order_id,region,product,month,sales,returns
0,104,West,Cable,2025-12-01,110,6.0
1,101,East,Wire,2025-11-01,120,5.0
2,106,West,Wire,2025-12-01,95,NaN
3,102,East,Cable,2025-12-01,150,4.0
4,105,West,Cable,2025-11-01,90,3.0
5,103,East,Wire,2025-11-01,80,2.0


In [3]:
#=========================================================
# Case 1) sort_values(): order rows by business columns
#=========================================================
case1 = (
    df.sort_values(by = ["month" , "sales" , "returns"] ,
        ascending = [True , False , True] , na_position = "last"
    )
)
case1

,order_id,region,product,month,sales,returns
1,101,East,Wire,2025-11-01,120,5.0
4,105,West,Cable,2025-11-01,90,3.0
5,103,East,Wire,2025-11-01,80,2.0
3,102,East,Cable,2025-12-01,150,4.0
0,104,West,Cable,2025-12-01,110,6.0
2,106,West,Wire,2025-12-01,95,NaN


In [4]:
#=======================================
# Case 2) sort_index(): order by index
#=======================================
tmp = df.set_index(["region" , "product" , "month"]).copy()
case2 = tmp.sort_index()
case2

order_id  sales  returns
region product month                               
East   Cable   2025-12-01       102    150      4.0
       Wire    2025-11-01       101    120      5.0
               2025-11-01       103     80      2.0
West   Cable   2025-11-01       105     90      3.0
               2025-12-01       104    110      6.0
       Wire    2025-12-01       106     95      NaN

In [5]:
#======================================
# Case 3) set_index() + reset_index()
#======================================
case3_set = df.set_index("order_id")
case3_set.head(3)

case3_reset = case3_set.reset_index()
case3_reset.head(3)

,region,product,month,sales,returns
order_id,,,,,
104,West,Cable,2025-12-01,110,6.0
101,East,Wire,2025-11-01,120,5.0
106,West,Wire,2025-12-01,95,NaN


,order_id,region,product,month,sales,returns
0,104,West,Cable,2025-12-01,110,6.0
1,101,East,Wire,2025-11-01,120,5.0
2,106,West,Wire,2025-12-01,95,NaN


In [6]:
#===============================================================
# Case 4) reindex(): enforce a canonical order + expose missing
#===============================================================
region_order = ["East" , "West" , "North"]  # North doesn't exist in data
by_region = df.groupby("region" , as_index = True)["sales"].sum()
case4 = by_region.reindex(region_order , fill_value = 0)
case4

region
East     350
West     295
North      0
Name: sales, dtype: int64

In [7]:
#===========================
# Case 5) Label alignment
#===========================
s1 = pd.Series([10 , 20 , 30] , index = ["A" , "B" , "C"])
s2 = pd.Series([100 , 200 , 300] , index = ["B" , "C" , "D"])

case5_add = s1 + s2
case5_add

case5_align_left , case5_align_right = s1.align(s2 , join = "inner")
case5_align_left + case5_align_right

A      NaN
B    120.0
C    230.0
D      NaN
dtype: float64

B    120
C    230
dtype: int64

In [8]:
#==============================================
# Case 6) MultiIndex slicing with IndexSlice
#==============================================
mi = df.set_index(["region" , "product" , "month"]).sort_index()
idx = pd.IndexSlice
case6 = mi.loc[idx["East" , "Cable" , :] , ["sales" , "returns"]]
case6

,,,sales,returns
region,product,month,,
East,Cable,2025-12-01,150,4.0
